In [1]:
import logging
from setting.db import SessionLocal
from llm_inference.base import LLMInterface
from graph.graph_knowledge_base import GraphKnowledgeBase
from entity_cond import EntityCond

logging.basicConfig(level=logging.INFO)

llm_client = LLMInterface("openai", "deepseek/deepseek-r1-distill-qwen-32b")
gkb = GraphKnowledgeBase(llm_client, "entities_150001", "relationships_150001", "chunks_150001")

ec = EntityCond(llm_client, gkb)

In [2]:
query = "What is the default timeout duration for TiCDC when executing operations on the downstream, and is there a parameter to control this timeout?"
model_kwargs = {
    "options": {
        "num_ctx": 8092,
        "num_gpu": 80,
        "num_predict": 10000,
        "temperature": 0.1,
    }
}
model_kwargs = {}

In [3]:
session = SessionLocal()
res = ec.analyze(session, query, **model_kwargs)

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:entity_cond:Initial analysis for What is the default timeout duration for TiCDC when executing operations on the downstream, and is there a parameter to control this timeout?: Analysis Result:
Reasoning: The user is inquiring about TiCDC's default timeout for downstream operations and whether it can be adjusted. Their core need is likely to optimize or troubleshoot TiCDC performance, especially in production environments where timely operations are crucial. Understanding the default settings and available parameters is essential for configuration and issue resolution. The search queries are designed to retrieve direct answers, configuration details, and practical examples to address these needs comprehensively.
Retrieval Queries:
    TiCDC default timeout downstream operations
    TiCDC operation timeout configuration parameter
    How to adjust TiCDC timeout for downstream tasks
    TiCD

In [4]:
print(res['final_answer'])

When executing operations on the downstream, TiCDC has a default timeout duration of **10 seconds** for dial, read, and write operations. This timeout behavior can be customized using the following parameters:
- `dial-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]
- `read-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]
- `write-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]


In [5]:
res

{'final_answer': 'When executing operations on the downstream, TiCDC has a default timeout duration of **10 seconds** for dial, read, and write operations. This timeout behavior can be customized using the following parameters:\n- `dial-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]\n- `read-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]\n- `write-timeout` [Source: https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]',
 'sources_adapted': ['https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr'],
 'map_results': [{'doc_link': 'https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr',
   'answer_segment': 'The default timeout duration for TiCDC when executing operations on the downstream is 10 seconds for both dial, read, and write operations. This can be controlled using the following parameters: `dial-timeout`, `read-timeout`, and `write-timeout`. [Source:https://docs.pingcap.com/tidb/v8.1/release-6.0.0-dmr]',
   'key_facts': ['The defaul